In [99]:
using HDF5, EzXML, Glob, Dates, Plots, LineSearches, LineSearches, JuMP, StatsBase, DataFrames, CSV
using BayesOpt, CurveFit
using CurveFit
using Symbolics
import NLopt
include("/Users/jjc/CSF/CSF Dynamics Scripts/readCSF.jl")

# datapath = "/Users/jjc/Documents/SSC/recs_june_2022"
datapath = "/Users/jjc/CSF/Recordings"
# datapath = "/Users/jjc/Documents/SSC/CSF Data/selected_recordings"
path = pwd();
savepath = "/Users/jjc/CSF/"
files = glob("*.hdf5", datapath);

df = DataFrame(filename=files, Rcsf=NaN, E=NaN, Ib=NaN, P0=NaN, mode=NaN, error=NaN, R2=NaN, Pss=NaN)

for f = 1:length(files)
# for f = 1141
    f in [278, 442, 676, 1043] ? (continue) : 0
    # global Data = Dict{String,Any}()
    filename = files[f]
    println(f, " ", filename)
    global Data = readCSF(filename) # Read all data from the hdf5 file
    try
        global Pm = Data["ICP"][Data["infusion_start_frame"]:Data["infusion_end_frame"]]
    catch
        continue
    end

    length(Pm) < 20 ? (continue) : 0

    # lowerbound = [0.01, 0.01, -10.0] # Lower boundary
    # upperbound = [50.0, 1.0, Data["P_b"]] # Upper boundary
    lowerbound = [0.01, 0.01, -10.0, -10] # Lower boundary
    upperbound = [50.0, 1.0, Data["P_b"], Data["P_b"]] # Upper boundary

    global Ib_lower = 0.01 # I_b range 
    global Ib_upper = 1.0 # I_b range
    global κ = 2
    global C = 10^3

    # BayesOpt params
    bkernel = "kMaternISO5"
    # bkernel = "kMaternARD1"
    bsctype = SC_MAP
    # bltype = L_MCMC
    bltype = 0
#
    # global Rcsf, E, P_0 = getModelBayes(lowerbound, upperbound, bkernel, bsctype, bltype)
    global Rcsf, E, P_0, Pss = getModelBayesPss(lowerbound, upperbound, bkernel, bsctype, bltype)
    # global Rcsf, E, Pss = getModelBayesStaticP0(lowerbound, upperbound, bkernel, bsctype, bltype)

    # NLOpt params
    optalg = :GN_DIRECT_L
    # optalg = :LD_LBFGS
    # optalg = :LN_NELDERMEAD
    # optalg = :LN_COBYLA
    # optalg = :GN_DIRECT_NOSCAL
    # optalg = :GN_CRS2_LM

    x0 = zeros(3)
    x0[1] = minimum([Data["Rcsf"], upperbound[1]])
    x0[2] = minimum([Data["E"], 1.0])
    x0[3] = maximum([Data["P_0"], lowerbound[3]])
    # x0[4] = minimum([0, lowerbound[4]])
 
    # global Rcsf, E, P_0 = getModelNL(lowerbound, upperbound, optalg, x0)
    # global Rcsf, E, P_0, Pss = getModelNLPss(lowerbound, upperbound, optalg, x0)
    # global Rcsf, E, Pss = getModelStaticP0(lowerbound, upperbound, optalg, x0)
    # P_0 = Data["P_0"]
    # global I_b = (Data["P_b"] - P_0) / Rcsf
    global I_b = (Data["P_b"] - Pss) / Rcsf

    # plot_model(I_b, E, P_0, Data["ICP"], 20, false)
    # R2, MSE = press_vol_curve(Rcsf, P_0)
    # savefig("/Users/jjc/Documents/SSC/recs_june_2022/GN_DIRECT_L_PV/" * filename[length(datapath)+2:end-5] * ".pdf")

    global df.Rcsf[f] = round(Rcsf, digits=3)
    global df.P0[f] = round(P_0, digits=3)
    global df.Ib[f] = round(I_b, digits=3)
    global df.E[f] = round(E, digits=3)
    global df.filename[f] = filename[length(datapath)+2:end]
    global df.error[f] = round(get_error_score(Rcsf, E, P_0), digits=3)
    global volRes, pressRes, fitted_curve, R2, MSE = press_vol_curve(Rcsf, P_0)
    global df.R2[f] = round(R2, digits=3)
    if @isdefined(Pss)
        global df.Pss[f] = round(Pss, digits=3)
    else
        global df.Pss[f] = round(P_0, digits=3)
    end
end
# CSV.write("/Users/jjc/CSF/Results/Results_BayesStaticP0_untruncPV.csv", df);
# println(Data["Rcsf"])
# plot_model(I_b, E, P_0, Data["ICP"], 20, false)
# plot(volRes, log.(pressRes), seriestype=:scatter, title=round(R2,digits=3), legend=false)
# plot(volRes, log.(pressRes), seriestype=:scatter)
# plot!(volRes, fitted_curve, legend=false, title=R2)

In [ ]:
Data

In [ ]:
infVec = zeros(Data["plateau_end"])
Data["infusion_end_frame"] > Data["plateau_end"] ? endframe = Data["plateau_end"] : Data["infusion_end_frame"]
infVec[Data["infusion_start_frame"]:endframe] .= 1

Rcsf = Data["Rcsf"]
E = Data["E"]
P_0 = Data["P_0"]

ΔP = Data["P_b"] - Data["P_0"]
I_b = ΔP / Rcsf
ICPm = ones(length(Data["ICP"])) .* Data["P_b"]
err = 0.0
for i = 1:endframe
    I_inf = Data["I_inf"]
    It = I_b + I_inf
    tᵢ = (i - 1) / 6
    Pᵢ = It * ΔP / (I_b + I_inf * exp(-E * It * tᵢ)) + P_0 + (I_inf * Data["Rn"])
    ICPm[i-1+Data["infusion_start_frame"]] = Pᵢ
    err += (Data["ICP"][i-1+Data["infusion_start_frame"]]-Pᵢ)^2
    (i>Data["plateau_start"] && i<Data["plateau_end"]) ? err+=(Data["P_p"] - (Data["P_b"] + Data["Rcsf"] * I_inf))^2 : 0
end

nmrse = 100 * sqrt(err) / endframe / abs(mean(Data["ICP"][1:endframe]))

ICPm[endframe:end] .= Data["P_b"]
plot(ICPm)
plot!(Data["ICP"])
vline!([endframe])
vline!([Data["infusion_start_frame"]])
println(nmrse)



In [ ]:
[1,2,3] \ [1,2,3]